In [9]:
import numpy as np
import math
import cv2
import pandas as pd
import mahotas
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from skimage.measure import regionprops
from skimage.transform import rescale, resize
from skimage.morphology import thin
from skimage.morphology import skeletonize
from skimage.morphology import medial_axis
from matplotlib.ticker import MultipleLocator
from utils import dataset_loader, my_utils, Operations, Moments, generate_point_cloud
import open3d as o3d
import time

In [10]:
surface     = ['FOCTS', 'PEG', 'ODS']
temperature = 24
load_NIPAM  = False
DATA_DIR    = "../../dataset/Dataset"
dataset_dictionary  = dataset_loader.get_dataset(DATA_DIR = DATA_DIR, temperature = temperature, surface =surface, load_NIPAM = load_NIPAM, Flag_zreverse = True)

In [11]:
a = dataset_dictionary['PEG'][0]

In [12]:
foct = o3d.geometry.PointCloud()
foct.points = o3d.utility.Vector3dVector(dataset_dictionary['FOCTS'][0])

peg = o3d.geometry.PointCloud()
peg.points = o3d.utility.Vector3dVector(dataset_dictionary['PEG'][0])

ods = o3d.geometry.PointCloud()
ods.points = o3d.utility.Vector3dVector(dataset_dictionary['ODS'][0])


In [ ]:
o3d.visualization.draw_geometries([foct])

In [ ]:
o3d.visualization.draw_geometries([peg])

In [ ]:
o3d.visualization.draw_geometries([ods])

In [ ]:
print("Load a ply point cloud, print it, and render it")
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dataset_dictionary['ODS'][0])

# o3d.visualization.draw_geometries([pcd])

print("Downsample the point cloud with a voxel of 0.02")
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.12)
o3d.visualization.draw_geometries([voxel_down_pcd])

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dataset_dictionary['ODS'][1])
tic = time.time()
keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd)
toc = 1000 * (time.time() - tic)
print("ISS Computation took {:.0f} [ms]".format(toc))
o3d.visualization.draw_geometries([keypoints])

In [ ]:
pcd = o3d.geometry.PointCloud()
xyz = dataset_dictionary['PEG'][1]
xyz /= xyz.max()
pcd.points = o3d.utility.Vector3dVector(xyz)
keypoints = pcd

# keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd)
alpha = 0.007
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(keypoints, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)


In [53]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dataset_dictionary['ODS'][0])
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals

pcd.estimate_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [55]:
### Visualization in form of animation
def custom_draw_geometry_with_rotation(pcd):

    def rotate_view(vis):
        ctr = vis.get_view_control()
        ctr.rotate(10.0, 0.0)
        return False

    o3d.visualization.draw_geometries_with_animation_callback([pcd],
                                                              rotate_view)
custom_draw_geometry_with_rotation(pcd)

In [ ]:
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.5, 0.5, 0.5])
keypoints.paint_uniform_color([1.0, 0.75, 0.0])
o3d.visualization.draw_geometries([keypoints, mesh], front=[0, 0, -1.0])

In [27]:
a =dataset_dictionary['ODS'][0]
a /= a.max()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(a)
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.008)
o3d.visualization.draw_geometries([voxel_down_pcd])



In [23]:
a

TrackedArray([[9.51430565e-01, 5.25440182e-01, 1.99229208e-02],
              [9.50857250e-01, 5.19207631e-01, 1.50607138e-02],
              [9.56260947e-01, 5.17615907e-01, 1.77944036e-02],
              ...,
              [9.72688106e-01, 5.29676971e-01, 5.81223008e-03],
              [9.76958743e-01, 5.22317852e-01, 2.30308622e-04],
              [9.70579058e-01, 5.21726041e-01, 8.66291180e-03]])

In [6]:
import open3d_tutorial as o3dtut
mesh = o3dtut.get_armadillo_mesh()
pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices

tic = time.time()
keypoints = o3d.geometry.keypoint.compute_iss_keypoints(pcd)
toc = 1000 * (time.time() - tic)
print("ISS Computation took {:.0f} [ms]".format(toc))

mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.5, 0.5, 0.5])
keypoints.paint_uniform_color([1.0, 0.75, 0.0])
o3d.visualization.draw_geometries([keypoints, mesh], front=[0, 0, -1.0])

ModuleNotFoundError: No module named 'open3d_tutorial'

In [16]:
#visualize all Foct samples
focts = []
for idx in range(len(dataset_dictionary['FOCTS'])):
    foct = o3d.geometry.PointCloud()
    foct.points = o3d.utility.Vector3dVector(dataset_dictionary['FOCTS'][idx])
    focts.append(foct)
o3d.visualization.draw_geometries(focts)


In [56]:
#visualize all Foct key points samples
focts = []
for idx in range(len(dataset_dictionary['FOCTS'])):
    foct = o3d.geometry.PointCloud()
    foct.points = o3d.utility.Vector3dVector(dataset_dictionary['FOCTS'][idx])
    keypoints = o3d.geometry.keypoint.compute_iss_keypoints(foct)
    focts.append(keypoints)
o3d.visualization.draw_geometries(focts)



In [21]:
#visualize all PEG samples
focts = []
for idx in range(len(dataset_dictionary['PEG'])):
    foct = o3d.geometry.PointCloud()
    foct.points = o3d.utility.Vector3dVector(dataset_dictionary['PEG'][idx])
    focts.append(foct)
o3d.visualization.draw_geometries(focts)

In [18]:
#visualize all PEG samples
focts = []
for idx in range(len(dataset_dictionary['ODS'])):
    foct = o3d.geometry.PointCloud()
    foct.points = o3d.utility.Vector3dVector(dataset_dictionary['ODS'][idx])
    focts.append(foct)
o3d.visualization.draw_geometries(focts)

In [7]:
vis = o3d.visualization.Visualizer()
vis.create_window()

# geometry is the point cloud used in your animaiton
geometry = o3d.geometry.PointCloud()
vis.add_geometry(geometry)

for i in range(1):
    # now modify the points of your geometry
    # you can use whatever method suits you best, this is just an example
    pcd.points = o3d.utility.Vector3dVector(dataset_dictionary['PEG'][i])
    geometry.points = pcd.points
    vis.update_geometry(geometry)
    vis.poll_events()
    vis.update_renderer()
